In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flights-and-airports-data/airports.csv
/kaggle/input/flights-and-airports-data/flights.csv
/kaggle/input/flights-and-airports-data/raw-flight-data.csv


In [2]:
pip install pyspark

     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 36.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=2344d2920ebe25cf1b46d7b995dd4292093d966b285e459e5e6320de6ce2c2b7
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql.types import *
from pyspark.sql import *

In [4]:
#Turning On Big machine
Spark=SparkSession.builder.master('local').appName("Warm Up Pyspark").getOrCreate()

In [5]:
#Load the dataset
dataflight=Spark.read.csv("../input/flights-and-airports-data/raw-flight-data.csv", inferSchema=True, header=True)
dataairport=Spark.read.csv("../input/flights-and-airports-data/airports.csv", inferSchema=True,header=True)
dataflight.show(5)
dataairport.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|   

In [6]:
dataflight.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



In [7]:
#Lets Join two table
FlightbyOrigin=dataflight.join(dataairport,dataflight.OriginAirportID==dataairport.airport_id).groupby('City').count()
FlightbyOrigin.show(5)

+--------------+-----+
|          City|count|
+--------------+-----+
|       Phoenix|90281|
|         Omaha|13537|
|Raleigh/Durham|28436|
|     Anchorage| 7777|
|        Dallas|19503|
+--------------+-----+
only showing top 5 rows



In [8]:
#Deal with Duplicate
print("Data All",dataflight.count())
print("Data Distinct", dataflight.dropDuplicates().count())
print("Data Duplicate", dataflight.count()-dataflight.dropDuplicates().count())

Data All 2719418
Data Distinct 2696983
Data Duplicate 22435


In [9]:
#Dealing With Missing Value
flightnomissingvalue=dataflight.dropDuplicates().dropna(how='any').count()
print("Missing Value:",dataflight.count()-flightnomissingvalue)

Missing Value: 46233


In [10]:
#Statistics 
FlightbyOrigin1=dataflight.join(dataairport,dataflight.OriginAirportID==dataairport.airport_id)
FlightbyOrigin1.describe('DepDelay','ArrDelay').show()

+-------+-----------------+-----------------+
|summary|         DepDelay|         ArrDelay|
+-------+-----------------+-----------------+
|  count|          2691974|          2690385|
|   mean|10.53686662649788| 6.63768791455498|
| stddev|36.09952806642889|38.64881489390084|
|    min|              -63|              -94|
|    max|             1863|             1845|
+-------+-----------------+-----------------+

